I wanted to throw in some additional notes about the modeler, for posterity. 

* In almost all cases, the NLSQ modeler is better than the MCMC one. I believe the primary reason is that MCMC is operating with an arm behind it's back. The liklihood the sampler evaluates is under the same assumptions that NLSQ is under, so it's no wonder it underperforms. It could be improved by a more nuanced noise model (which actually is one of the advantages of MCMC is that you can use any model you like). I tried using a mixture model and Bayesian Outlier detection, neither worked. The principle of outlier detection is that each pixel has a nuisance parameter associated with it which determines how likely that pixel is to be background. It is a bit overkill for this case because whether or not an individual pixel is not random nor independant of the others. All the signal will be clustered together in a bright spot! It didn't work because having nearly 1,000 parameters ate up all my RAM, and because the logarithm of the liklihood doesn't factorize super easily, and numerical errors creep up to be too difficult to work with. 

* A note on parameter selection. In the ideal case, the Maximum A Posteriori (MAP) estimator is the mean of the sample chain. This is, in effect, marginalizing over all other paramters to get an estimator for one specific parameter. This still holds true, but when the distribution becomes extremely complex that value may not actually be a practically good number. For example, if it's heavily bimodal and the mean is smack in the middle, the value chosen isn't gonna work very well. In practice, I found either the median or mode was a better estimator, depending. I usually prefered the median. Problems could still pop up, but it was much more robust to outliers. It would occasionally be more biased to the center of the image, though.  For certain sets of variables that are very correlated, a more advanced scheme would be better. I go into more detail with positions, below. 

* I left one more branch open (though it's a little behind on updates) which is the allMovingCenters branch. A true MoG model will allow each Gaussian to move independantly of the others. However, this became difficult to implement in practice. Parameter selection in general was difficult, but parameter selection on positions after the first one was nigh impossible to do. One can imagine 2 "bright spots" on a diagonal, at (10,10) and (20,20). When marginalized over y, x will be chosen to be either 10 or 20, and vice-versa as well. But, unless it's very well sampled, (10,20) or (20,10) could be chosen by mistake! All parameters could in theory suffer from this, but it almost always arose with multiple positions. With one position, like there is in the current implementation, the convergence at the brightest spot was usually really clear. It was the additional Guassians moving around the image that would cause trouble. Like I said, perhaps some parameter reduction or clustering would make for a better selection. 

* Lastly, a note on degeneracy. There is a constraint that the amplitudes have to be in sorted order. That's because the model has a built-in degeneracy of N! for the number of Gaussians N. This is because I could switch Gaussians 1-2-3 to 2-1-3 or 3-2-1, etc. and get the same model. This degeneracy makes parameter selection a nightmare, because all parameters would converge to the same distribution. For awhile I was doing some really nightmarish stuff to solve it, until I figured out the ordering solution. It effectively partitions the parameter space and excises the degeneracies. I am fairly certain that adding another cut would possibly screw up the modeler, but I'm not sure. 